In [4]:
import requests
import psycopg2
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from src.databases import Postgresql

# Criando um lista de skus e de urls

In [5]:
#Sku
lst_sku=[]
for i in range(1,90):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'
    if i % 10 == 0:
        print(f'Raspando pagina {i}')

    resultado = requests.get(PAGE)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ProductDisplay-name'})
    sku = np.array([elem.a['data-product-sku'] for elem in html_data])
    
    for elem in html_data:
        lst_sku.append(elem.a['data-product-sku'])

Raspando pagina 10
Raspando pagina 20
Raspando pagina 30
Raspando pagina 40
Raspando pagina 50
Raspando pagina 60
Raspando pagina 70
Raspando pagina 80


In [6]:
lista = []
lista_vinhos = []

for i in range(1,90):

    r = requests.get(f'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=false&pn={i}&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS')
    soup = BeautifulSoup(r.content,'html.parser')
    container = soup.find_all('div',{'class':'ProductDisplay-name'})
    for col in container: # contém todos os vinhos
        for art in col.find_all('a',href=True): #vinhos em cada coluna
            lista = 'https://www.wine.com.br{}'.format(art['href'])
            lista_vinhos.append(lista)

In [7]:
lista_vinhos[0:3]

['https://www.wine.com.br/vinhos/maison-le-star-rose-on-ice/prod18479.html',
 'https://www.wine.com.br/vinhos/champagne-montaudon-brut/prod3170.html',
 'https://www.wine.com.br/vinhos/ropiteau-pinot-noir-2017/prod22807.html']

# Extraindo Variáveis

In [8]:
#script
tipo = []
uva = []
country = []
region = []
vinicola = []
teor = []
amadurecimento = []
classificacao = []
safra = []
visual = []
ofativo = []
gustativo = []
temperatura = []
potencial_guarda = []

for i, j in enumerate(lista_vinhos):
    prods_url = j
    resultado = requests.get(prods_url)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'Right'}, limit=1)
    html_data_country = bs.find_all('li',{'class':'PageHeader-tag'}, limit=1)
    html_data_region = bs.find_all('label',{'class':"Breadcrumb-item"})
    html_data_vinicola = bs.find_all('div',{'class':"col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--winery"})
    html_data_teor = bs.find_all('div',{'class':"col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--alcoholic_strength"})
    html_data_amadure = bs.find_all('div',{'class':"col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--ageing"})
    html_data_safra = bs.find_all('div',{'class': 'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--harvest'})
    html_data_class = bs.find_all('div',{'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--classification'})
    html_data_visual = bs.find_all('div',{'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--appearance'})
    html_data_ofativo = bs.find_all('div',{'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--aroma'})
    html_data_gustativo = bs.find_all('div',{'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--taste'})
    html_data_temp_guarda = bs.find_all('div',{'col-xs-8 col-sm-12 TechnicalDetails-description TechnicalDetails-description--temperature'})
 
    for elem in html_data:
        tipo.append({'sku':lst_sku[i], 'tipo':elem.dt.string})
        uva.append({'sku':lst_sku[i],'uva':elem.dd.string})
        
    for elem in html_data_country:
        country.append({'sku':lst_sku[i],'country':elem.string})
    
    for elem in html_data_region:
        region.append({'sku':lst_sku[i], 'region':elem.string})
        
    for elem in html_data_vinicola:
        vinicola.append({'sku':lst_sku[i],'vinicola':elem.dd.string})
        
    for elem in html_data_teor:
        teor.append({'sku':lst_sku[i], 'teor_alcoolico':elem.dd.string})
    
    for elem in html_data_amadure:
        amadurecimento.append({'sku':lst_sku[i],'amadurecimento':elem.dd.string})
        
    for elem in html_data_class:
        classificacao.append({'sku':lst_sku[i],'classificacao':elem.dd.string})
        
    for elem in html_data_safra:
        safra.append({'sku':lst_sku[i],'safra':elem.dd.string})
    
    for elem in html_data_visual:
        visual.append({'sku':lst_sku[i],'visual':elem.dd.string})
    
    for elem in html_data_ofativo:
        ofativo.append({'sku':lst_sku[i],'ofativo':elem.dd.string})
        
    for elem in html_data_gustativo:
        gustativo.append({'sku':lst_sku[i],'gustativo':elem.dd.string})
    
    for elem in html_data_temp_guarda:
        if elem.dt.string == 'Temperatura de serviço':
            temperatura.append({'sku':lst_sku[i],'temperatura':elem.dd.string})
        elif elem.dt.string == 'Potencial de guarda':
            potencial_guarda.append({'sku':lst_sku[i],'potencial_guarda':elem.dd.string})


#Dataset principal
#df_temp = np.column_stack((lst_sku,tipo, uva,country,region,vinicola,teor, amadurecimento, classificacao, safra,visual,ofativo,gustativo,temperatura,potencial_guarda))
#dataset_1 = pd.DataFrame(df_temp, columns=['sku','tipo','uva','country','region','vinicola','teor_alcolico', 'amadurecimento', 'classificacao', 'safra',
                                            #'visual','ofativo','gustativo','temperatura','potencial_guarda'])

#Gerando os dataframes
dataset_tipo = pd.DataFrame(tipo)
dataset_tipo.set_index('sku', inplace= True)

dataset_uva = pd.DataFrame(uva)
dataset_uva.set_index('sku', inplace= True)

dataset_country = pd.DataFrame(country)
dataset_country.set_index('sku', inplace= True)

dataset_region = pd.DataFrame(region)
dataset_region.set_index('sku', inplace= True)

dataset_vinicola = pd.DataFrame(vinicola)
dataset_vinicola.set_index('sku', inplace= True)

dataset_teor = pd.DataFrame(teor)
dataset_teor.set_index('sku', inplace= True)

dataset_amadurecimento = pd.DataFrame(amadurecimento)
dataset_amadurecimento.set_index('sku', inplace= True)

dataset_classificacao = pd.DataFrame(classificacao)
dataset_classificacao.set_index('sku', inplace= True)

dataset_safra = pd.DataFrame(safra)
dataset_safra.set_index('sku', inplace= True)

dataset_visual = pd.DataFrame(visual)
dataset_visual.set_index('sku', inplace= True)

dataset_ofativo = pd.DataFrame(ofativo)
dataset_ofativo.set_index('sku', inplace= True)

dataset_gustativo = pd.DataFrame(gustativo)
dataset_gustativo.set_index('sku', inplace= True)

dataset_temperatura = pd.DataFrame(temperatura)
dataset_temperatura.set_index('sku', inplace= True)

dataset_potencial_guarda = pd.DataFrame(potencial_guarda)
dataset_potencial_guarda.set_index('sku', inplace= True)

In [9]:
df = dataset_tipo.join([dataset_uva, dataset_country, dataset_region, 
                dataset_vinicola, dataset_teor, dataset_amadurecimento, dataset_classificacao, 
                dataset_safra, dataset_visual, dataset_ofativo, dataset_gustativo, dataset_temperatura, dataset_potencial_guarda], how = 'inner')
df = df.reset_index()

In [12]:
query = """ CREATE TABLE ficha_tecnicav1 ( sku TEXT,
                                            tipo               TEXT,
                                            uva                TEXT,
                                            country            TEXT,
                                            region             TEXT,
                                            vinicola           TEXT,
                                            teor_alcoolico     TEXT,
                                            amadurecimento     TEXT,
                                            classificacao      TEXT,
                                            safra              TEXT,
                                            visual             TEXT,
                                            ofativo            TEXT,
                                            gustativo          TEXT,
                                            temperatura        TEXT,
                                            potencial_guarda   TEXT)  """



bd = Postgresql()
bd.create_table(query=query,user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.insert_data(df = df, tabela = ' ficha_tecnicav1',user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada
